# GDELT Demo project showcase

This is the simple presentation of my findings and a bit about the skills involved in getting there. For gory details, see [README.md](https://github.com/reed9999/gdelt-demo/blob/master/README.md).

## Findings and visualizations

This project is still in early stages, but this is where I will present findings.

### Research questions
To get interesting answers, though, I need good questions. This is what makes this project more than just an extended SQL tutorial. In this kind of research there's going to be some iteration in RQs, because things you think *a priori* might be interesting turn out not to be, and vice versa. So this is largely a running list of ideas, .

1. **Media tone and stability** - As I understand, the Rosenstein score is a measure of how each kind of action promotes or erodes stability. The tone score is something about the tone of the media coverage of the event. How much do they correlate?

1. **Actor affinity by dyad** - Do some actor dyads consistently produce higher scores (Rosenstein or media tone) than others? I'd think this would be a trivial "Yes" because relationships between allies should produce more positive news than those between adversaries. So this is exploring the obvious, but a good sanity check to make sure I'm understanding the nature of this data.

  1. **Asymmetrical relationships** - Just brainstorming. If X threatens Y more than Y threatens X that's also significant and interesting.
  
1. **Longitudinal time series questions** - This dataset is so rich that it could support a lot of this.

1. **Clustering countries or other actors** - If I want to polish up those *k*-means clustering skills, there's probably all kinds of variables I could derive ("number of threats made", "number of overtures to negotiation", "general tone of media coverage", etc.).

1. **Network graph analysis** - I don't actually know much about this topic, but clearly this data would support a lot of inferences about how actors build their networks. 



## Skills demonstrated

* **General quant research**: This will become evident in time. For now the best demonstration is the RQs immediately above.
* **Hive QL and SQL**: Several exploratory queries: 
  * For now the best examples are those having to do with the Goldstein scale column, see [queries/goldsteinscale](https://github.com/reed9999/gdelt-demo/tree/master/queries/goldsteinscale). Most of those should run as SQL against a local MySQL.
  
* **Spark** (via PySpark): See [queries/spark-sql](https://github.com/reed9999/gdelt-demo/tree/master/queries/spark-sql).
* **AWS setup** (S3, EMR cluster creation)
  * **via CLI**: See [automation/scripts](https://github.com/reed9999/gdelt-demo/tree/master/automation/scripts).
  * **via boto3**: See [automation](https://github.com/reed9999/gdelt-demo/tree/master/automation).